In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from citipy import citipy
from datetime import datetime
import time
import requests
from config import weather_api_keys


In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

# Use the print() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

85.6230206273209 121.96769619430682
33.82698378042656 2.0000562999824183
79.8852484087829 -71.18743511259034
51.04961705119388 -132.4235406217848
24.999722464760197 29.42758948708945
64.90593999622146 -28.717944102262095
-23.79639826720296 -137.2249156106186
-56.36563250138536 -37.37354756909221
-83.1585632679512 -96.34292522081451
15.18297488648922 -53.050437697515136
-76.63488087661386 -8.475683010248474
-6.5532330177207 -166.56427168867685
-33.78613684147055 8.926079124176653
-52.29197008862853 -147.2576884952876
20.446102326468747 6.7226453239158275
-46.97710454551061 -117.74577781200324
-2.9754169934074355 -88.02822627391427
-24.826271587808748 -35.83783234126628
-50.1215974555414 -94.01606240132999
65.73740282838884 126.01563959695886
83.59404737312985 -51.30358379433079
-60.408889869551246 24.261695949963297
23.020090529273688 -150.43270746336663
64.68991864633023 -126.25384698844559
6.4581644490465635 -28.033352285482152
59.48732592053446 -42.00651125724218
-78.30747571430503 -

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)


744

### Most of this is copied straight from WeatherPy.ipynb

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_keys

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
#lets not do this again.....

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]   
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Current Description": city_weather_description,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | tiksi
Processing Record 2 of Set 1 | aflu
City not found. Skipping...
Processing Record 3 of Set 1 | qaanaaq
Processing Record 4 of Set 1 | prince rupert
Processing Record 5 of Set 1 | tahta
Processing Record 6 of Set 1 | olafsvik
Processing Record 7 of Set 1 | rikitea
Processing Record 8 of Set 1 | mar del plata
Processing Record 9 of Set 1 | punta arenas
Processing Record 10 of Set 1 | bathsheba
Processing Record 11 of Set 1 | cape town
Processing Record 12 of Set 1 | samusu
City not found. Skipping...
Processing Record 13 of Set 1 | saldanha
Processing Record 14 of Set 1 | mataura
Processing Record 15 of Set 1 | arlit
Processing Record 16 of Set 1 | san cristobal
Processing Record 17 of Set 1 | sao joao da barra
Processing Record 18 of Set 1 | castro
Processing Record 19 of Set 1 | sangar
Processing Record 20 of Set 1 | upernavik
Processing Record 21 of Set 1 | bredasdorp
Processing Record 22 

In [6]:
len(city_data)
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
0,Tiksi,71.6872,128.8694,39.54,overcast clouds,90,99,7.58,RU
1,Qaanaaq,77.4840,-69.3632,49.44,overcast clouds,65,90,8.77,GL
2,Prince Rupert,54.3161,-130.3201,55.49,light rain,96,100,11.50,CA
3,Tahta,26.7693,31.5021,86.86,clear sky,25,0,10.67,EG
4,Olafsvik,64.8945,-23.7142,51.57,overcast clouds,84,100,14.76,IS
5,Rikitea,-23.1203,-134.9692,68.11,light rain,77,100,7.74,PF
6,Mar Del Plata,-38.0023,-57.5575,52.09,light intensity drizzle,95,100,8.01,AR
7,Punta Arenas,-53.1500,-70.9167,33.82,few clouds,93,20,10.36,CL
8,Bathsheba,13.2167,-59.5167,86.61,broken clouds,86,75,19.57,BB
9,Cape Town,-33.9258,18.4232,62.38,clear sky,82,0,3.00,ZA


In [8]:
columns = ['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Current Description']
city_data_df = city_data_df[columns]
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Tiksi,RU,71.6872,128.8694,39.54,90,99,7.58,overcast clouds
1,Qaanaaq,GL,77.4840,-69.3632,49.44,65,90,8.77,overcast clouds
2,Prince Rupert,CA,54.3161,-130.3201,55.49,96,100,11.50,light rain
3,Tahta,EG,26.7693,31.5021,86.86,25,0,10.67,clear sky
4,Olafsvik,IS,64.8945,-23.7142,51.57,84,100,14.76,overcast clouds


In [9]:
city_data_df.dtypes

City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [10]:
# Create the output File (CSV)
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the city_data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")